In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image
import os
import os.path
from os import path
import tensorflow as tf
from keras.models import Sequential, Model 
from keras.layers import *
import keras.backend as k
import keras.utils
from keras import optimizers as opt
from sklearn.model_selection import train_test_split
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from skimage import measure
import math

print(k.common.image_dim_ordering())

k.common.set_image_dim_ordering('tf')

gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

Using TensorFlow backend.


tf


In [2]:
def normalize(arr):
    arrMin = np.amin(arr)
    arrMax = np.amax(arr)
    print(arrMin)
    print(arrMax)
    if arrMax != 0:
        arr = np.subtract(arr,arrMin)
        #print(arr)
        arrMax = np.amax(arr)
        arr = np.divide(arr,arrMax)
        #print(arr)
    else:
        print("error, max value is zero")
    print("normalized")
    return arr

In [3]:
#criar batch (entrada de cada iteração)
import keras.utils
import scipy.ndimage

batch_size=3
max_rotation_angle = 10
max_shift = 0.2
max_zoom = 0.2

class BatchGenerator(keras.utils.Sequence):
    
    def __init__(self,
                 x_set,
                 y_set,
                 batch_size,
                 image_dimensions=(128, 128, 128),
                 shuffle=True,
                 n_channels=1,
                 n_classes=2):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
        self.image_dimensions = image_dimensions
        print("Generator created for image size: {}".format(self.image_dimensions))
        self.shuffle = shuffle
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.number_of_images = self.x.shape[0]
        self.indices = np.arange(self.number_of_images)
        if self.shuffle == True:
            np.random.shuffle(self.indices)
            
        #print(self.x.shape)
        #print(self.y.shape)
            
    def __len__(self):
        return int(np.floor(self.number_of_images / self.batch_size))
    
    def on_epoch_end(self):
        self.indices = np.arange(self.number_of_images)
        if self.shuffle == True:
            np.random.shuffle(self.indices)
    
    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]
        x = np.empty((self.batch_size, *self.image_dimensions, self.n_channels))
        y = np.empty((self.batch_size, *self.image_dimensions))
        #print(batch_indices)
        
        for i in range(self.batch_size):
            flip_flag = np.random.randint(2)
            #print(i)
            if flip_flag == 1:
                x[i,:,:,:,:] = np.flip(self.x[batch_indices[i],:,:,:,:], axis=0)
                y[i,:,:,:]   = np.flip(self.y[batch_indices[i],:,:,:,0], axis=0)
            else:
                x[i,:,:,:,:] = self.x[batch_indices[i],:,:,:,:]
                y[i,:,:,:]   = self.y[batch_indices[i],:,:,:,0]
            
        # Rotations
        
        x_rot = np.copy(x)
        y_rot = np.copy(y)
            
        for i in range(self.batch_size):
            #print("aug",i)
            angle_x = np.random.randint(-max_rotation_angle, max_rotation_angle)
            x_rot[i,:,:,:,:] = scipy.ndimage.interpolation.rotate(
                x[i,:,:,:,:], angle_x, (1,2), False, mode="constant", cval=0, order=0)
            y_rot[i,:,:,:] = scipy.ndimage.interpolation.rotate(
                y[i,:,:,:], angle_x, (1,2), False, mode="constant", cval=0, order=0)
        
            #angle_y = np.random.randint(-max_rotation_angle, max_rotation_angle)
            #x_rot = scipy.ndimage.interpolation.rotate(x, angle_y, (0,2), False, mode="constant", cval=0, order=0)
            #y_rot = scipy.ndimage.interpolation.rotate(y, angle_y, (0,2), False, mode="constant", cval=0, order=0)

            #angle_z = np.random.randint(-max_rotation_angle, max_rotation_angle)
            #x_rot = scipy.ndimage.interpolation.rotate(x, angle_z, (0,1), False, mode="constant", cval=0, order=0)
            #y_rot = scipy.ndimage.interpolation.rotate(y, angle_z, (0,1), False, mode="constant", cval=0, order=0)
        
        # shift
        
        shift = np.random.uniform(-max_shift, max_shift, size=5)
        shift[0] = 0.0
        shift[4] = 0.0
        # x_shift = scipy.ndimage.interpolation.shift(x_rot, shift)
        # y_shift = scipy.ndimage.interpolation.shift(y_rot, shift[:4])
        
        # make sure values are between 0 and 1
        
        # x_aug = np.clip(x_shift, 0.0, 1.0)
        # y_aug = np.clip(y_shift, 0.0, 1.0)
        
        x_aug = np.clip(x_rot, 0.0, 1.0)
        y_aug = np.clip(y_rot, 0.0, 1.0)
        
        # convert segmentation to one-hot encoding
        
        y_onehot = keras.utils.to_categorical(y_aug, self.n_classes)

        return x_aug, y_onehot

In [4]:
#criar rede
num_classes = 2
filter_multiplier = 20

def nvidia_unet(input_size=128, num_classes=num_classes):
    input_ = Input((input_size, input_size, input_size, 1))
    skips = []
    output = input_
    c = num_classes
    
    num_layers = int(np.floor(np.log2(input_size)))
    down_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    down_filter_numbers = np.zeros([num_layers], dtype=int)
    up_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    up_filter_numbers = np.zeros([num_layers], dtype=int)
    
    for layer_index in range(num_layers):
        down_conv_kernel_sizes[layer_index] = int(3)
        down_filter_numbers[layer_index] = int( (layer_index + 1) * filter_multiplier + num_classes )
        up_conv_kernel_sizes[layer_index] = int(4)
        up_filter_numbers[layer_index] = int( (num_layers - layer_index - 1) * filter_multiplier + num_classes )
    
    print("Number of layers:       {}".format(num_layers))
    print("Filters in layers down: {}".format(down_filter_numbers))
    print("Filters in layers up:   {}".format(up_filter_numbers))
    
    for shape, filters in zip(down_conv_kernel_sizes, down_filter_numbers):
        skips.append(output)
        output= Conv3D(filters, (shape, shape, shape), strides=2, padding="same", activation="relu")(output)
        
    for shape, filters in zip(up_conv_kernel_sizes, up_filter_numbers):
        output = keras.layers.UpSampling3D()(output)
        skip_output = skips.pop()
        output = concatenate([output, skip_output], axis=4)

        if filters != num_classes:
            output = Conv3D(filters, (shape, shape, shape), activation="relu", padding="same")(output)
            output = BatchNormalization(momentum=.9)(output)
        else:
            output = Conv3D(filters, (shape, shape, shape), activation="sigmoid", padding="same")(output)
    
    assert len(skips) == 0
    return Model([input_], [output])

In [5]:
rootPath = "/home/tallys/3dunet/"
nrrdPath = "/home/tallys/3dunet/img/"
segPath = "/home/tallys/3dunet/mask"
outputPath = "/home/tallys/3dunet/output"

nrrdFilePaths = []
nrrdFileNames = []
preNormalizedFilePaths = []
normalizedFilePaths = []
normalizedFileNames = []

labelFilePaths = []

for root, dirs, files in os.walk(nrrdPath):
        for filename in files:
            if filename.endswith("normalized.npy") or filename.endswith("5dim.npy"):
                continue
            elif filename.endswith(".npy"):
                path=os.path.join(root,filename)
                #print("found",path)
                a = np.load(path)
                nrrdFilePaths.append(path)
                #very messy, but is necessary to weed out the normalized images
                if filename.endswith("normalized.npy") or filename.endswith("5dim.npy"):
                    continue
                elif filename.endswith(".npy"):
                    noExtension = filename[:-4]
                    nrrdFileNames.append(noExtension)
                    normalizedFileNames.append(noExtension)
                    newExtension = noExtension + '_normalized.npy'
                    outPutFileName = os.path.join(root,newExtension)
                    normalizedFilePaths.append(outPutFileName)
                    
                    preNewExtension = noExtension + '_pnormalized.npy'
                    preOutputFileName = os.path.join(root,preNewExtension)
                    preNormalizedFilePaths.append(preOutputFileName)
                
print(len(nrrdFilePaths),"nrrds found")
print(len(preNormalizedFilePaths),"downsized file paths created")
print(len(normalizedFilePaths),"normalized file paths created")

4 nrrds found
4 downsized file paths created
4 normalized file paths created


In [6]:
downsizedLabelFilePaths=[]
labelFilePaths=[]
for root, dirs, files in os.walk(segPath):
        for filename in files:
            if filename.endswith("Label.npy"):                
                noExtension = filename[:-4]
                newExtension = noExtension + '_downsized.npy'
                outPutFileName = os.path.join(root,newExtension)
                downsizedLabelFilePaths.append(outPutFileName)
                
                path=os.path.join(root,filename)
                labelFilePaths.append(path)
print(len(downsizedLabelFilePaths),"downsized label paths created")

4 downsized label paths created


In [19]:
import os
import os.path
from os import path
from scipy.ndimage import zoom

#preNormalized AKA downsized
imageZResize = []
nnnn=0
skippedImageDownsizeCount=0
imageDownsizeCount=0

for filePath in preNormalizedFilePaths:
    if path.exists(filePath):
        skippedImageDownsizeCount+=1
        
    elif not path.exists(filePath):
        file=nrrdFilePaths[nnnn]
        print(file)
        arr = np.load(file)
        arr = zoom(arr, (0.25,0.25,0.25))
        
        shape=arr.shape
        zAxis=(shape[0])
        resize=128/zAxis
        imageZResize.append(resize)
        arr=zoom(arr, (resize,1,1))
        print(arr.shape)
        
        np.save(filePath,arr)
        imageDownsizeCount+=1
    nnnn+=1

print(skippedImageDownsizeCount,"images skipped")
print(imageDownsizeCount,"images downsized")

4 images skipped
0 images downsized


In [8]:
import os
import os.path
from os import path

labelZResize=[]
nnnnn=0
skippedLabelDownsizeCount=0
labelDownsizeCount=0

for filePath in downsizedLabelFilePaths:
    #print(e)
    if path.exists(filePath):
        skippedLabelDownsizeCount+=1
        
    elif not path.exists(filePath):
        file=labelFilePaths[nnnnn]
        print(file)
        arr = np.load(file)
        arr = zoom(arr, (0.25,0.25,0.25))
        
        shape=arr.shape
        zAxis=(shape[0])
        resize=128/zAxis
        labelZResize.append(resize)
        arr=zoom(arr, (resize,1,1))
        print(arr.shape)
        
        np.save(filePath,arr)
        labelDownsizeCount+=1
    nnnnn+=1
    
print(skippedLabelDownsizeCount,"labels skipped")
print(labelDownsizeCount,"labels downsized")

4 labels skipped
0 labels downsized


In [9]:
skippedNormalizedCount=0
normalizedCount=0

n=0
for filePath in normalizedFilePaths:
    #print(e)
    if path.exists(filePath):
        skippedNormalizedCount+=1
        
    elif not path.exists(filePath):
        file=preNormalizedFilePaths[n]
        arr = np.load(file)
        normalized = normalize(arr)
        np.save(filePath,normalized)
        normalizedCount+=1
    n+=1
    
print(skippedNormalizedCount,"scans skipped")
print(normalizedCount,"scans normalized")

4 scans skipped
0 scans normalized


In [18]:
eps = 2
smooth = 1
lam = 1

def dice_coef(y_true, y_pred):
    y_true_f = k.flatten(y_true)
    y_pred_f = k.flatten(y_pred)
    intersection = k.sum(y_true_f * y_pred_f)
    
    return (2 * intersection + smooth) / (k.sum(y_true_f) + k.sum(y_pred_f) + smooth)

def custom_binary_crossentropy(y_true,y_pred):
    #experimental binary crossentropy loss metric
    #which takes into account the amount of islands
    #not working and therefore
    #unused at the time of writing
    bc = keras.losses.binary_crossentropy(y_true, y_pred)
    islands = num_islands(bc,0)
    loss = bc*(K.log(islands+eps))**lam
    return (loss)

In [16]:
image5Dim = []
image5DimPaths = []
imageSkippedCount = 0
image5DimmedCount = 0

n1=0

for filePath in normalizedFilePaths:
    filePath = filePath[:-4]
    filePath = filePath + '_5dim.npy'
    image5DimPaths.append(filePath)

for filePath in image5DimPaths:
    if path.exists(filePath):
        arr=np.load(filePath)
        image5Dim.append(arr)
        imageSkippedCount+=1
        
    elif not path.exists(filePath):
        file=normalizedFilePaths[n1]
        print(file)
        arr=np.load(file)
        arr=arr[...,np.newaxis]
        np.save(image5DimPaths[n1],arr)
        
        print(np.shape(arr))
        image5Dim.append(arr)
        image5DimmedCount+=1
    n1+=1
    
print(imageSkippedCount,"scans skipped")
print(image5DimmedCount,"scans given an extra dimension")

4 scans skipped
0 scans given an extra dimension


In [20]:
label5Dim = []
label5DimPaths = []
label5DimmedCount = 0
labelSkippedCount = 0

n2=0

for filePath in downsizedLabelFilePaths:
    filePath = filePath[:-4]
    filePath = filePath + '_5dim.npy'
    label5DimPaths.append(filePath)

for filePath in label5DimPaths:
    if path.exists(filePath):
        arr=np.load(filePath)
        label5Dim.append(arr)
        labelSkippedCount+=1
        
    elif not path.exists(filePath):
        file=downsizedLabelFilePaths[n2]
        print(file)
        arr=np.load(file)
        arr=arr[...,np.newaxis]
        np.save(label5DimPaths[n2],arr)
        
        print(np.shape(arr))
        label5Dim.append(arr)
        label5DimmedCount +=1
    n2+=1

print(labelSkippedCount,"labels skipped")
print(label5DimmedCount,"labels given an extra dimension")

4 labels skipped
0 labels given an extra dimension


In [14]:
size=np.array(image5Dim).shape[1:4]
#size=np.ndarray(image5Dim).shape[1:4]
X_train, X_test, y_train, y_test = train_test_split (image5Dim, label5Dim, test_size = 0.01)

print(np.shape(X_train))
print(np.shape(y_train))

/home/tallys/anaconda3/envs/3dunet/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


ValueError: could not broadcast input array from shape (128,128,115,1) into shape (128,128)

In [ ]:
max_learning_rate = 0.001
min_learning_rate = 0.0001
num_epochs = 20

learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

#set initial values as very easy to beat
#so the first iteration will qualify for all of these
prev_max_acc = np.full((num_epochs), 0, dtype=int)
prev_min_acc = np.full((num_epochs), 1, dtype=int)

prev_max_loss = np.full((num_epochs), 0, dtype=int)
prev_min_loss = np.full((num_epochs), 1, dtype=int)
accuracyLog = []


In [ ]:
import gc
gc.collect()
tf.compat.v1.InteractiveSession.close(session)

In [ ]:
for iteration in range((len(image5Dim))):
    
    import tensorflow as tf

    k.common.set_image_dim_ordering('tf')
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

    X_test2 = image5Dim[iteration]
    X_test2 = X_test2[np.newaxis,...]
    newImage5Dim = np.array(np.delete(image5Dim,iteration,axis=0))
    
    y_test2 = label5Dim[iteration]
    y_test2 = y_test2[np.newaxis,...]
    newLabel5Dim = np.array(np.delete(label5Dim,iteration,axis=0))
    
    
    trainingData = BatchGenerator(np.array(newImage5Dim),np.array(newLabel5Dim),image_dimensions=(size),batch_size=3)
    validationData = BatchGenerator(np.array(X_test2),np.array(y_test2),image_dimensions=(size),batch_size=1)
    
    model = nvidia_unet(size[0], num_classes)
    
    model.compile(optimizer=keras.optimizers.adam(lr=max_learning_rate, decay=learning_rate_decay),
               loss= "binary_crossentropy",
               metrics=[dice_coef])
    
    history = model.fit_generator(trainingData,
                        epochs=num_epochs,
                        verbose=2)
    
    score = model.evaluate_generator(validationData)
    
    print(score,"on loop",iteration)
    
    accuracyLog.append(score)
    
    accuracy = (score[1])
    loss = (score[0])
    
    #this is purely for generation of graphs depicting the accuracy/loss trends
    if history.history['dice_coef'][num_epochs-1] > prev_max_acc[num_epochs-1]:
        prev_max_acc = history.history['dice_coef']
        print("new max training accuracy:",history.history['dice_coef'][num_epochs-1],"max curve updated")
        
    if history.history['dice_coef'][num_epochs-1] < prev_min_acc[num_epochs-1]:
        prev_min_acc = history.history['dice_coef']
        print("new min training accuracy:",history.history['dice_coef'][num_epochs-1],"min curve updated")
        
    if history.history['loss'][num_epochs-1] > prev_max_loss[num_epochs-1]:
        prev_max_loss = history.history['loss']
        print("new max training loss:",history.history['loss'][num_epochs-1],"max loss curve updated")
        
    if history.history['loss'][num_epochs-1] < prev_min_loss[num_epochs-1]:
        prev_min_loss = history.history['loss']
        print("new min training loss:",history.history['loss'][num_epochs-1],"min loss curve updated")
    
    del history
    del model
    
    #it occasionally does not clear on the first attempt
    for e in range(5):
        gc.collect()
    k.clear_session()
    tf.InteractiveSession.close(session)
    
    #while it clogs the output, this is done so that
    #if memory runs out mid validation, it is easy to see
    #the most up to date accuracy log
    print(accuracyLog)